## 第12章 pandas高级应用

In [1]:
import pandas as pd
import numpy as np

### 12.1 分类数据

表中的⼀列通常会有重复的包含不同值的⼩集合的情况。我们已
经学过了unique和value_counts，它们可以从数组提取出不同的
值，并分别计算频率：

In [5]:
values = pd.Series(['apple', 'orange', 'apple',
                    'apple'] * 2)
values


0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [6]:
pd.unique(values)


array(['apple', 'orange'], dtype=object)

In [7]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

许多数据系统（数据仓库、统计计算或其它应⽤）都发展出了特
定的表征重复值的⽅法，以进⾏⾼效的存储和计算。在数据仓库
中，最好的⽅法是使⽤所谓的包含不同值得维表(Dimension
Table)，将主要的参数存储为引⽤维表整数键

In [8]:
values = pd.Series([0, 1, 0, 0] * 2)

In [9]:
dim = pd.Series(['apple', 'orange'])

In [10]:
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [11]:
dim

0     apple
1    orange
dtype: object

可以使⽤take⽅法存储原始的字符串Series：

In [13]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

这种⽤整数表示的⽅法称为分类或字典编码表示法。不同值得数组称为分类、字典或数据级。本书中，我们使⽤分类的说法。表示分类的整数值称为分类编码或简单地称为编码。

分类表示可以在进⾏分析时⼤⼤的提⾼性能。你也可以在保持编码不变的情况下，对分类进⾏转换。⼀些相对简单的转变例⼦包括

- 重命名分类
- 加入一个新的分类，不改变已经存在的分类的顺序或位置

### pandas的分类类型

pandas 有一个特殊的分类类型，用于保存使用整数分类表示法的数据

In [14]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,12,0.492389
1,1,orange,10,2.076093
2,2,apple,14,0.498868
3,3,apple,4,2.109604
4,4,apple,11,2.979223
5,5,orange,4,1.384068
6,6,apple,12,3.094579
7,7,apple,12,2.509482


这里，df['fruit']是一个Python字符串对象的数组。我们可以通过调用它，将它转变为分类：

In [15]:
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

fruit_cat的值不是NumPy数组，⽽是⼀个pandas.Categorical实
例

In [17]:
c = fruit_cat.values
type(c)

pandas.core.arrays.categorical.Categorical

分类对象有categories和codes属性

In [18]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [19]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

你可将DataFrame的列通过分配转换结果，转换为分类：

In [21]:
df['fruit'] = df['fruit'].astype('category')
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

你还可以从其它Python序列直接创建pandas.Categorical

In [22]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

如果你已经从其它源获得了分类编码，你还可以使⽤from_codes
构造器：

In [23]:

categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

与显示指定不同，分类变换不认定指定的分类顺序。因此取决于
输⼊数据的顺序， categories数组的顺序会不同。当使⽤
from_codes或其它的构造器时，你可以指定分类⼀个有意义的顺
序：

In [26]:
ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)
ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

输出[foo < bar < baz]指明‘foo’位于‘bar’的前⾯，以此类推。⽆序
的分类实例可以通过as_ordered排序：

In [29]:
my_cats_2.as_ordered()

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]